In [ ]:
#https://stackoverflow.com/questions/27934885/how-to-hide-code-from-cells-in-ipython-notebook-visualized-with-nbviewer
#cool code for a button to hide code 
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [173]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import tabulate
import statistical_tools as s_tools
import statsmodels.api as sm 
from statsmodels.formula.api import glm 
#plotting with seaborn
#https://cmdlinetips.com/2019/02/how-to-make-histogram-in-python-with-pandas-and-seaborn/

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

891

In [174]:
train = train[train['Age'].notna()]
train = train[train['Pclass'].notna()]
train = train[train['Sex'].notna()]

In [211]:
from statsmodels.stats.outliers_influence import variance_inflation_factor 

# Get variables for which to compute VIF and add intercept term 
#Pclass == 1 and Sex == male are the reference factors for those categories - it is redundant to put them in and actually causes errors

X = train[['Age', 'Parch', 'SibSp']].copy()
X['class 2'] = np.where(train['Pclass']== 2, 1, 0)
X['class 3'] = np.where(train['Pclass']== 3, 1, 0)
X['F'] = np.where(train['Sex']== 'female', 1, 0)
X['Intercept'] = 1 

  

# Compute and view VIF 
vif = pd.DataFrame() 
vif["variables"] = X.columns 

vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])] 

  

# View results using print 

print(vif) 

   variables        VIF
0        Age   1.311466
1      Parch   1.241783
2      SibSp   1.263703
3    class 2   1.546774
4    class 3   1.770723
5          F   1.114458
6  Intercept  15.839099


In [222]:
# Define model formula 

train_half = train[0:450].copy()
train_2nd_half = train[450:len(train)].copy()

formula = 'Survived ~ C(Sex, Treatment(\'male\')) + C(Pclass, Treatment(1)) + Age + Parch + SibSp' 

model = glm(formula, data = train_half, family = sm.families.Binomial()).fit() 

print(model.summary()) 
pred_glm = model.predict(train_2nd_half) 
#predicted_survive = (pred_glm >= 0.5)
train_2nd_half['expected_survive'] = pred_glm
 

                 Generalized Linear Model Regression Results                  
Dep. Variable:               Survived   No. Observations:                  450
Model:                            GLM   Df Residuals:                      443
Model Family:                Binomial   Df Model:                            6
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -205.83
Date:                Tue, 12 May 2020   Deviance:                       411.66
Time:                        15:41:27   Pearson chi2:                     472.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In

In [216]:
#train_2nd_half[0:30]

In [240]:
# Compute estimated probabilities for GLM model: pred_glm

 
l = list()

for i in np.linspace(0.0, 1, 40):
    predicted_survive = (pred_glm >= i)
    #print(predicted_survive)
    #break
    #print(predicted_survive)
    #print(train_2nd_half['Survived'])
    num_correct = (predicted_survive.astype(bool)) == (train_2nd_half['Survived'].astype(bool))
    s = pd.DataFrame({'pred': predicted_survive,
                  'surv': train_2nd_half['Survived'],
                     'AND': num_correct})
    #print(s[0:20])
    #break
    l.append([i, sum(num_correct)])

m = 0
pr = 0
for i in l:
    if i[1] > m:
        m = i[1]
        pr = i[0]
        
print('Best threshold: {}'.format(pr))



Best threshold: 0.5641025641025641


In [248]:
len(test)

418

In [249]:
# RUN MODEL ON TEST DATA

formula = 'Survived ~ C(Sex, Treatment(\'male\')) + C(Pclass, Treatment(1)) + Age + Parch + SibSp' 

model = glm(formula, data = train, family = sm.families.Binomial()).fit() 

print(model.summary()) 


                 Generalized Linear Model Regression Results                  
Dep. Variable:               Survived   No. Observations:                  714
Model:                            GLM   Df Residuals:                      707
Model Family:                Binomial   Df Model:                            6
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -318.23
Date:                Tue, 12 May 2020   Deviance:                       636.46
Time:                        16:04:51   Pearson chi2:                     748.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In

In [250]:
pred_glm = model.predict(test) 
predicted_survive = (pred_glm >= pr)
test['Survived'] = predicted_survive.astype(int)
test_file = test[['PassengerId', 'Survived']]
test_file.to_csv('submission.csv')

In [ ]:
'0.784 score on kaggle - not bad!''